In [3]:
import datasets
from datasets import inspect_dataset, load_dataset_builder

inspect_dataset("wmt19", "path/to/scripts")
builder = load_dataset_builder(
    "path/to/scripts/wmt_utils.py",
    language_pair=("fr", "de"),
    subsets={
        datasets.Split.TRAIN: ["commoncrawl_frde"],
        datasets.Split.VALIDATION: ["euelections_dev2019"],
    },
)

# Standard version
builder.download_and_prepare()
ds = builder.as_dataset()

Using custom data configuration fr-de-70c1275b0f74c5d7


The processing script for dataset wmt19 can be inspected at /home/zerui/Documents/Transformer/path/to/scripts. The main class is in /home/zerui/.cache/huggingface/modules/datasets_modules/datasets/wmt19/29e210fae5690e843cae5dc43b53db36c4e02f927db50cd5235a22ab42dde90a. You can modify this processing script and use it with `datasets.load_dataset("/home/zerui/Documents/Transformer/path/to/scripts")`.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

In [1]:
import torch
import numpy as np

In [6]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.tril(np.ones(attn_shape)).astype('uint8')
    return torch.from_numpy(subsequent_mask)

In [7]:
subsequent_mask(10)

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], dtype=torch.uint8)

In [8]:
causal_mask = torch.tril(torch.ones(10, 10, dtype=bool))

In [10]:
causal_mask & False

tensor([[False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False]])